# Baseball Analytics Project

load necessary packages.

In [1]:
import requests
import lxml.html as lh
import pandas as pd

The teams with their numbers that show up in urls

In [90]:
teams =["BC", "GOC", "GRC", "HU", "INWU", "MAR", "MVNU", "SAU", "SFIN", "TAYL"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

Scrape the Webpage(s).

In [84]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/IndividualStats.aspx?association=10&sg=MBA&conference=NAIMBA1_CROSS&team=' +
        str(num) + '&sea=NAIMBA_2020' for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the contents of the website under doc
docs = [lh.fromstring(page.content) for page in pages]
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = [doc.xpath('//tr') for doc in docs]

`tr_elements` stores rows of the html page between `<tr>` and `</tr>`.  We found the three tables by running the code below and looking for groups of rows that all have the same length.  The three tables ended up being

1.  Batting at rows 49 through 76
2.  Pitching at rows 77 through 90
3.  Fielding at rows 91 through 118

In [93]:
#look_at_tr = [len(T) for T in tr_elements[2]]
#for i in range(len(look_at_tr)):
#    print(i, len(tr_elements[2][i]))

Subset to only the rows we need for each dataframe.

In [86]:
B_elements = [T_els[49:77] for T_els in tr_elements] #Batting
P_elements = [T_els[78:91] for T_els in tr_elements] #Pitching
F_elements = [T_els[92:119] for T_els in tr_elements] #Fielding


Below is a function for checking if data is an integer, and if it's not, checking if it is a float.

In [87]:
def nice_data(d):
    #If we can turn it into an integer, then do that
    try:
        d=int(d)
    except:
    #If it's not an integer, it might be a float
        try:
            d = float(d)
        except:
            pass
    return(d)

In [96]:
B_Data = [[[
    nice_data(d.text_content()) #get the text content and turn it into numbers if possible
    for d in row.iterchildren() #for each element in the row
    ]
    for row in B_els #for each row in the list of rows
    ]
    for B_els in B_elements #for each list of rows corresponding to each team
]
dfb = [pd.DataFrame(columns = D[0], data = D[1:]) for D in B_Data]
dfb[2]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,...,SLG%,BB,HBP,SO,GDP,OB%,SF,SH,SB,SBA
0,"Wagner, Ezra",3,2,0.500,2,0,1,0,0,0,...,0.500,2,0,0,0,0.750,0,0,0,0
1,"Griffin, Chris",14,4,0.415,53,15,22,3,1,2,...,0.623,6,2,12,1,0.492,0,0,2,3
2,"Rich, Alex",16,6,0.333,57,11,19,1,0,0,...,0.351,7,4,10,1,0.441,0,0,2,3
3,"Anderson, David",16,4,0.315,54,4,17,4,1,0,...,0.426,0,0,6,3,0.315,0,0,2,2
4,"Haney, Houston",13,6,0.294,17,2,5,2,0,0,...,0.412,4,0,5,1,0.429,0,0,0,1
5,"Mullet, Jaron",16,3,0.269,26,7,7,2,0,0,...,0.346,5,0,11,1,0.387,0,0,0,0
6,"Pawlik, Jeff",16,6,0.255,55,8,14,4,0,0,...,0.327,10,1,17,0,0.379,0,0,0,0
7,"Newkirk, Sam",14,4,0.245,49,6,12,3,0,3,...,0.490,5,1,12,0,0.321,1,0,3,3
8,"Brooks, Ethan",16,6,0.226,53,4,12,4,0,0,...,0.302,7,0,18,1,0.317,0,0,0,2
9,"Harris, Xavier",16,6,0.213,47,5,10,3,0,0,...,0.277,5,6,9,0,0.362,0,2,1,1


In [11]:
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements2[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Pitching
2 ERA
3 W
4 L
5 GP
6 GS
7 CG
8 SHO
9 CBO
10 SV
11 IP
12 H
13 R
14 ER
15 BB
16 SO
17 2B
18 3B
19 HR
20 TBF
21 B/Avg
22 WP
23 HBP
24 BK
25 SFA
26 SHA


In [12]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements2)):
    #T is our j'th row
    T=tr_elements2[j]
    #If row is not of size 26, the //tr data is not from our table 
    if len(T)!=26:
        break
        #i is the index of our column
    i=0
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [13]:
[len(C) for (title,C) in col]

[13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13]

In [14]:
Dict={title:column for (title,column) in col}
dfp=pd.DataFrame(Dict)

In [15]:
dfp

,Pitching,ERA,W,L,GP,GS,CG,SHO,CBO,SV,...,2B,3B,HR,TBF,B/Avg,WP,HBP,BK,SFA,SHA
0,"Harris, Xavier",0.00,1,0,2,1,0,0,0,0,...,0,0,0,24,0.000,3,1,1,0,0
1,"Decker, Jake",2.84,1,0,4,0,0,0,0,0,...,0,0,0,49,0.140,0,1,1,0,0
2,"Wilhelmi, Bret",3.38,0,1,5,2,0,0,0,1,...,0,0,0,35,0.115,0,1,0,0,0
3,"Haney, Houston",3.46,1,2,4,4,0,0,0,0,...,2,2,0,104,0.255,0,2,0,1,0
4,"Stoltzfus, Nick",4.22,2,0,7,0,0,0,0,0,...,0,0,0,50,0.256,2,3,0,0,0
5,"Meyer, Mason",6.10,0,0,7,0,0,0,0,0,...,0,1,1,54,0.293,1,1,0,0,0
6,"Clark, Tanner",6.52,0,2,5,3,0,0,0,0,...,0,0,2,85,0.222,2,1,0,0,0
7,"Schumacher, Hunter",8.22,0,2,6,3,0,0,0,1,...,0,0,0,40,0.368,0,1,1,1,0
8,"Koch, Kameron",8.44,1,0,2,0,0,0,0,0,...,3,0,1,25,0.217,0,0,0,0,0
9,"Peterson, Ike",8.49,0,1,5,3,0,0,0,0,...,0,0,0,62,0.396,1,1,0,0,0


In [16]:
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements3[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Fielding
2 GP
3 GS
4 C
5 PO
6 A
7 E
8 FLD%
9 DP
10 TP
11 SBA
12 RCS
13 SBA%
14 PB
15 CI
16 OBS


In [17]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements3)):
    #T is our j'th row
    T=tr_elements3[j]
    #If row is not of size 16, the //tr data is not from our table 
    if len(T)!=16:
        break
        #i is the index of our column
    i=0
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [18]:
[len(C) for (title,C) in col]

[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27]

In [19]:
Dict={title:column for (title,column) in col}
dff=pd.DataFrame(Dict)

In [20]:
dff

,Fielding,GP,GS,C,PO,A,E,FLD%,DP,TP,SBA,RCS,SBA%,PB,CI,OBS
0,"Carr, Colin",2,2,11,10,1,0,1.000,0,0,1,0,1.000,3,0,0
1,"Peterson, Ike",5,3,1,0,1,0,1.000,0,0,0,0,0.000,0,0,0
2,"Wilhelmi, Bret",5,2,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0
3,"Stoltzfus, Nick",7,0,2,0,2,0,1.000,1,0,0,0,0.000,0,0,0
4,"Schumacher, Hunter",6,3,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0
5,"Meyer, Mason",7,0,5,0,5,0,1.000,0,0,0,0,0.000,0,0,0
6,"Clark, Tanner",5,3,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0
7,"Bertke, Ryan",8,1,4,4,0,0,1.000,0,0,0,0,0.000,0,0,0
8,"Haney, Houston",13,6,25,10,15,0,1.000,5,0,0,0,0.000,0,0,0
9,"Mullet, Jaron",16,3,7,7,0,0,1.000,0,0,0,0,0.000,0,0,0


In [21]:
dff["C/GP"] = round(dff["C"] / dff["GP"], 3)

In [22]:
dff

,Fielding,GP,GS,C,PO,A,E,FLD%,DP,TP,SBA,RCS,SBA%,PB,CI,OBS,C/GP
0,"Carr, Colin",2,2,11,10,1,0,1.000,0,0,1,0,1.000,3,0,0,5.500
1,"Peterson, Ike",5,3,1,0,1,0,1.000,0,0,0,0,0.000,0,0,0,0.200
2,"Wilhelmi, Bret",5,2,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0,0.200
3,"Stoltzfus, Nick",7,0,2,0,2,0,1.000,1,0,0,0,0.000,0,0,0,0.286
4,"Schumacher, Hunter",6,3,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0,0.167
5,"Meyer, Mason",7,0,5,0,5,0,1.000,0,0,0,0,0.000,0,0,0,0.714
6,"Clark, Tanner",5,3,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0,0.200
7,"Bertke, Ryan",8,1,4,4,0,0,1.000,0,0,0,0,0.000,0,0,0,0.500
8,"Haney, Houston",13,6,25,10,15,0,1.000,5,0,0,0,0.000,0,0,0,1.923
9,"Mullet, Jaron",16,3,7,7,0,0,1.000,0,0,0,0,0.000,0,0,0,0.438


In [23]:
dfb

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,...,SLG%,BB,HBP,SO,GDP,OB%,SF,SH,SB,SBA
0,"Wagner, Ezra",3,2,0.500,2,0,1,0,0,0,...,0.500,2,0,0,0,0.750,0,0,0,0
1,"Griffin, Chris",14,4,0.415,53,15,22,3,1,2,...,0.623,6,2,12,1,0.492,0,0,2,3
2,"Rich, Alex",16,6,0.333,57,11,19,1,0,0,...,0.351,7,4,10,1,0.441,0,0,2,3
3,"Anderson, David",16,4,0.315,54,4,17,4,1,0,...,0.426,0,0,6,3,0.315,0,0,2,2
4,"Haney, Houston",13,6,0.294,17,2,5,2,0,0,...,0.412,4,0,5,1,0.429,0,0,0,1
5,"Mullet, Jaron",16,3,0.269,26,7,7,2,0,0,...,0.346,5,0,11,1,0.387,0,0,0,0
6,"Pawlik, Jeff",16,6,0.255,55,8,14,4,0,0,...,0.327,10,1,17,0,0.379,0,0,0,0
7,"Newkirk, Sam",14,4,0.245,49,6,12,3,0,3,...,0.490,5,1,12,0,0.321,1,0,3,3
8,"Brooks, Ethan",16,6,0.226,53,4,12,4,0,0,...,0.302,7,0,18,1,0.317,0,0,0,2
9,"Harris, Xavier",16,6,0.213,47,5,10,3,0,0,...,0.277,5,6,9,0,0.362,0,2,1,1


In [31]:
# Caught stealing
dfb["CS"] = round(dfb["SBA"] - dfb["SB"], 3)
# Plate Appearances
dfb["PA"] = round(dfb["AB"] + dfb["SF"] + dfb["SH"] + dfb["BB"] + dfb["HBP"])

In [39]:
# OPS
dfb["OB%"] = dfb["OB%"].astype(float)
dfb["SLG%"] = dfb["SLG%"].astype(float)
dfb["OPS"] = dfb["OB%"] + dfb["SLG%"]

In [45]:
# Singles
dfb["1B"] = dfb["H"] - dfb["2B"] - dfb["3B"] - dfb["HR"]
# Runs Created
dfb["RC"] = round((dfb["H"] + dfb["BB"]) * dfb["TB"] / (dfb["AB"] + dfb["BB"]), 3)
# POP
dfb["AVG"] = dfb["AVG"].astype(float)
dfb["POP"] = dfb["OPS"] + dfb["AVG"]
dfb

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,...,SF,SH,SB,SBA,CS,OPS,PA,1B,RC,POP
0,"Wagner, Ezra",3,2,0.500,2,0,1,0,0,0,...,0,0,0,0,0,1.250,4,1,0.750,1.750
1,"Griffin, Chris",14,4,0.415,53,15,22,3,1,2,...,0,0,2,3,1,1.115,61,16,15.661,1.530
2,"Rich, Alex",16,6,0.333,57,11,19,1,0,0,...,0,0,2,3,1,0.792,68,18,8.125,1.125
3,"Anderson, David",16,4,0.315,54,4,17,4,1,0,...,0,0,2,2,0,0.741,54,12,7.241,1.056
4,"Haney, Houston",13,6,0.294,17,2,5,2,0,0,...,0,0,0,1,1,0.841,21,3,3.000,1.135
5,"Mullet, Jaron",16,3,0.269,26,7,7,2,0,0,...,0,0,0,0,0,0.733,31,5,3.484,1.002
6,"Pawlik, Jeff",16,6,0.255,55,8,14,4,0,0,...,0,0,0,0,0,0.706,66,10,6.646,0.961
7,"Newkirk, Sam",14,4,0.245,49,6,12,3,0,3,...,1,0,3,3,0,0.811,56,6,7.556,1.056
8,"Brooks, Ethan",16,6,0.226,53,4,12,4,0,0,...,0,0,0,2,2,0.619,60,8,5.067,0.845
9,"Harris, Xavier",16,6,0.213,47,5,10,3,0,0,...,0,2,1,1,0,0.639,60,7,3.750,0.852


In [ ]:
# hi